In [1]:
import pandas as pd 
import numpy as  np 
from sklearn.metrics import accuracy_score
import pickle

In [2]:
test = pd.read_csv("ApartmentRentPrediction_Milestone2.csv")

In [3]:
columns =["amenities","bedrooms","bathrooms","square_feet","latitude","longitude",
          "cityname","state","RentCategory"]
test = test[columns]

In [4]:
with open("mode_values.pickle", "rb") as f:
    loaded_mode_mean_values = pickle.load(f)

pets_allowed_mode= loaded_mode_mean_values["pets_allowed_mode"]
cityname_mode= loaded_mode_mean_values["cityname_mode"]
state_mode = loaded_mode_mean_values["state_mode"]
bedroom_mean = loaded_mode_mean_values["bedrooms_mean"]
bathroom_mean = loaded_mode_mean_values["bathrooms_mean"]
long_mean = loaded_mode_mean_values["longitude_mean"]
latit_mean = loaded_mode_mean_values["latitude_mean"]


test["amenities"].fillna("Parking", inplace=True)
test["cityname"].fillna(cityname_mode, inplace=True)
test["state"].fillna(state_mode, inplace=True)
test["bedrooms"].fillna(bedroom_mean, inplace=True)
test["bathrooms"].fillna(bathroom_mean, inplace=True)
test["longitude"].fillna(long_mean, inplace=True)
test["latitude"].fillna(latit_mean, inplace=True)

In [5]:
test.columns

Index(['amenities', 'bedrooms', 'bathrooms', 'square_feet', 'latitude',
       'longitude', 'cityname', 'state', 'RentCategory'],
      dtype='object')

In [6]:
with open('top_15_words.pkl', 'rb') as f:
    top_15_words = pickle.load(f)

In [7]:
for i, row in test.iterrows():
    text = row['amenities']
    found = False  # Initialize found flag for each row
    a = 0
    for word in top_15_words:
        if word in text:
            found = True
            text = text.replace(word, str(top_15_words.index(word) + 1))
            a = a+1
    
    text  = str(a)
    if not found:
        text = str(0)
        
    test.at[i, 'amenities'] = text

In [8]:
test["amenities"]=test["amenities"].astype(int)

In [9]:
def in_area(row):
    if (-123 <= row['longitude'] <= -114) and (30 <= row['latitude'] <= 38):
        return 1
    else:
        return 0

In [10]:
test['in_area'] = test.apply(in_area, axis=1)

In [11]:
with open('label_encoders.pickle', 'rb') as f:
    loaded_label_encoders = pickle.load(f)
    
columns = ['cityname', 'state','RentCategory']

for column in columns:
    encoder = loaded_label_encoders[column]
#    print(encoder.classes_)
    test[column + '_encoded'] = encoder.transform(test[column])

    

test["city_encoded"]= test["cityname_encoded"]
test.drop(columns = "cityname_encoded",axis=1,inplace=True)
# test[loaded_label_encoders[columns[0]]] = encoder.transform(test[columns[0]])

# test[loaded_label_encoders[columns[1]]] = encoder.transform(test[columns[1]])

# test[loaded_label_encoders[columns[2]]] = encoder.transform(test[columns[1]])

In [12]:
test.drop(columns=["cityname","state","RentCategory"],axis=1,inplace=True)

In [13]:
test = test[['amenities', 'bathrooms', 'bedrooms', 'square_feet', 'latitude', 'longitude', 'in_area', 'city_encoded', 'state_encoded', 'RentCategory_encoded']]
test.columns = ['amenities', 'bathrooms', 'bedrooms', 'square_feet', 'latitude', 'longitude', 'in_area', 'city_encoded', 'state_encoded', 'RentCategory_encoded']

In [14]:

numerical_columns = ['bathrooms', 'bedrooms', 'square_feet', 'latitude', 'longitude']

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

test[numerical_columns] = scaler.transform(test[numerical_columns])

In [15]:
with open('random_forest_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
    
X=test.drop(columns="RentCategory_encoded",axis=1)
y=test["RentCategory_encoded"]

In [16]:
predictions = loaded_model.predict(X)

accuracy_score(y,predictions)

0.916